<a href="https://colab.research.google.com/github/bok-h22/TIL-202303/blob/master/SKT_koBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade jax jaxlib

In [ ]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
# # 네이버 영화 리뷰 분류하기 BERT 계열 모델로는 89~90%까지 도달가능하다고 알려져 있음
!pip install transformers
!pip install sentencepiece
!pip install tensorflow_addons

In [ ]:
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import urllib.request
import tensorflow_addons as tfa

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not insta

In [ ]:
# 로그가 너무 많이 떠서 colab이 멎는 현상이 있음
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
# ! pip install pymysql

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data = pd.read_csv("/content/drive/MyDrive/FinalProject_데이터엔지니어링24회차/NLP/review_by_year.csv", encoding='utf-8')
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/reviews_0721.csv", encoding='utf-8')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30679 entries, 0 to 30678
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   corp         30679 non-null  object
 1   stock_code   30679 non-null  int64 
 2   cleand_up    30679 non-null  object
 3   cleand_down  30679 non-null  object
 4   year         30679 non-null  int64 
 5   sector       30679 non-null  object
 6   rank         30679 non-null  object
 7   label        30679 non-null  int64 
dtypes: int64(3), object(5)
memory usage: 1.9+ MB


In [ ]:
data.reset_index(inplace=True)
data=data.rename(columns={'cleand_up' : 'document', 'index':'id'})
data.dropna(inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data[['id','document', 'label']], test_size=0.25, stratify=data['label'].to_list())
train_data

,id,document,label
4470,4470,요새 권장휴가라면서 강제느낌이 강함 점심은 월급에 포함됨 젊은 꼰대들이 많다,5
22079,22079,유통업이어서 실생활 접점이 많아 직원할인 좋음어디가서 자랑하기는 괜찮음,6
29107,29107,에폭시 세계 위 성장해나가는 회사 돈 적당히 주고 지역 연고지,4
18200,18200,잘되면 내 덕 안되면 네 탓하는 사람들이 많다내로남불의 극치인 사람들도 많다이런 사...,4
21224,21224,휴가는 그래도 많이 쓸수있다업무 자체의 난이도는 어렵지가 않다사실 이보다 큰 장점이...,3
...,...,...,...
15584,15584,월급 꽤 많이 준다 월급 나올 때 행복하다취재 협조가 잘 되는 편큰 사고 안치면 대...,6
25139,25139,사내문화가 좋지 않아서 멘탈을 기를 슨 있다 밥이 맛 없어서 음식에 대한 감사함을 ...,6
10965,10965,주말이 없음나가야만 되는사람들이 있음그사람들은 절대 안나감,6
12142,12142,합리적인 유연근무제 전통적 si영역에서는 높은 전문성을 가졌습니다 infra clo...,7


In [ ]:
print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력

훈련용 리뷰 개수 : 23009


In [ ]:
print('검증용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력

검증용 리뷰 개수 : 7670


In [ ]:
import urllib.request #kobert_transformers/tokenization_kobert.py
urllib.request.urlretrieve("https://raw.githubusercontent.com/monologg/KoBERT-NER/master/tokenization_kobert.py", filename="tokenization_kobert.py")

('tokenization_kobert.py', <http.client.HTTPMessage at 0x799816479390>)

In [ ]:
from transformers import BertModel
from tokenization_kobert import KoBertTokenizer
# KoBertTokenizer 파일 안에 from transformers import PreTrainedTokenizer가 이미 되어있음

model = BertModel.from_pretrained('monologg/kobert')
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}



loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/model.safetensors
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at monologg/kobert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


loading file tokenizer_78b3253a26.model from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/tokenizer_78b3253a26.model
loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/config.json
Model config BertConfig {
  "_name_or_path": "monologg/kobert",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_d

In [ ]:
def convert_data(data_df):
    global tokenizer

    SEQ_LEN = 128 #SEQ_LEN : 버트에 들어갈 인풋의 길이 500

    tokens, masks, segments, targets = [], [], [], []

    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN].iloc[i], max_length=SEQ_LEN, pad_to_max_length=True)

        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        pad_count = token.count(1)
        mask = [1]*(SEQ_LEN-pad_count) + [0]*pad_count

        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN].iloc[i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 100 #500
BATCH_SIZE = 32
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "document"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "label"

In [ ]:
# train 데이터를 버트 인풋에 맞게 변환
train_X, train_y = load_data(train_data)

  0%|          | 0/23009 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 23009/23009 [00:05<00:00, 3844.51it/s]


In [ ]:
# 문장 순서 바꾸려면 마침표 토큰 54를 기준으로 순서를 바꾸기!!!!
train_X[1].shape

(23009, 128)

In [ ]:
# 훈련 성능을 검증한 test 데이터를 버트 인풋에 맞게 변환
test_X, test_y = load_data(test_data)

100%|██████████| 7670/7670 [00:01<00:00, 4040.34it/s]


In [ ]:
SEQ_LEN = 128

In [ ]:
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}



loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/pytorch_model.bin
Loading PyTorch weights from /root/.cache/huggingface/hub/models--monologg--kobert/snapshots/30941062c0f3dde73b246468f449f2448c7694bc/pytorch_model.bin
PyTorch checkpoint contains 92,186,880 parameters
Loaded 92,186,880 parameters in the TF 2.0 model.
All PyTorch model weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


bert_outputs[0]은 모든 시점의 BERT의 최종 벡터. 문장 길이가 64일 경우에는 64 x 768  
bert_outputs[1]은 CLS 시점의 BERT의 벡터

In [ ]:
bert_outputs = bert_outputs[1]

In [ ]:
bert_outputs.shape

TensorShape([None, 768])

In [ ]:
!pip install focal_loss

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
from tensorflow_addons.metrics import F1Score
from focal_loss import SparseCategoricalFocalLoss

loss_func = SparseCategoricalFocalLoss(gamma=2)
f1_score = F1Score(num_classes=9, average='weighted', name='f1_score')

In [ ]:
from keras.layers import Dense, Dropout
from keras import Model
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=-0.1, patience=10)

In [ ]:
# Rectified Adam 옵티마이저 사용
# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(learning_rate=1.0e-3, total_steps = 64*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

In [ ]:
#flattened = tf.keras.layers.Flatten()(bert_outputs)
x = tf.keras.layers.Dropout(0.5)(bert_outputs)
# x = Dense(128, activation='tanh')(x)
# x = Dense(64, activation='tanh')(x)
# x = Dense(32, activation='tanh')(x)
sentiment_output = Dense(9, activation='softmax', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(x)

sentiment_model = Model(inputs=[token_inputs, mask_inputs, segment_inputs], outputs=sentiment_output)
sentiment_model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics = ['accuracy']) #, f1_score
sentiment_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 128)]        0           []                               
                                                                                                  
 input_segment (InputLayer)     [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  92186880    ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]',      

In [ ]:
history = sentiment_model.fit(train_X,
                              train_y,
                              epochs=100,
                              batch_size=64,
                              validation_data=(test_X, test_y))

Epoch 1/100
360/360 [==============================] - 2123s 6s/step - loss: 2.0276 - accuracy: 0.2107 - val_loss: 2.0040 - val_accuracy: 0.2236
Epoch 2/100
360/360 [==============================] - 2083s 6s/step - loss: 2.0111 - accuracy: 0.2180 - val_loss: 2.0036 - val_accuracy: 0.2236
Epoch 3/100
360/360 [==============================] - 2084s 6s/step - loss: 2.0114 - accuracy: 0.2179 - val_loss: 2.0043 - val_accuracy: 0.2236
Epoch 4/100
134/360 [==========>...................] - ETA: 19:51 - loss: 2.0029 - accuracy: 0.2192

In [ ]:
! pip install tensorflow_decision_forests

In [ ]:
import tensorflow_decision_forests as tfdf

model = tfdf.keras.RandomForestModel()
model.fit(train_X)

NotFoundError: ignored

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []

    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
        num_zeros = token.count(1)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

In [ ]:
test_set = predict_load_data(test)

In [ ]:
test_set

In [ ]:
preds = sentiment_model.predict(test_set)

In [ ]:
# 부정이면 0, 긍정이면 1 출력
preds

In [ ]:
from sklearn.metrics import classification_report
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

In [ ]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True)

    num_zeros = token.count(0)
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def movie_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()

    if predict_answer == 0:
      print("(부정 확률 : %.2f) 부정적인 영화 평가입니다." % (1-predict_value))
    elif predict_answer == 1:
      print("(긍정 확률 : %.2f) 긍정적인 영화 평가입니다." % predict_value)

In [ ]:
def create_downstream(bert_outputs, fs_data):
  dim_reudction_dense = Dense(128, activation='tanh')(bert_outputs)
  concat_vector = tf.concat([dim_reduction_dense, fs_data], axis=1)
  x = Dense(128, activation='relu')(concat_vector)
  x = Dense(64, activation='relu')(x)
  x = Dense(32, activation='relu')(x)
  y = Dense(9, activation='softmax')(x)

  return y